# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data and Setup

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### Polynomial Conversion

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [5]:
poly_features = polynomial_converter.fit_transform(X)

### Train | Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [8]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# help(StandardScaler)

In [9]:
scaler = StandardScaler()

In [10]:
scaler.fit(X_train)

StandardScaler()

In [11]:
X_train = scaler.transform(X_train)

In [12]:
X_test = scaler.transform(X_test)

## Ridge Regression

Make sure to view video lectures for full explanation of Ridge Regression and choosing an alpha.

In [13]:
from sklearn.linear_model import Ridge

In [14]:
ridge_model = Ridge(alpha=10)

In [15]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [16]:
test_predictions = ridge_model.predict(X_test)

In [17]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [18]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [19]:
MAE

0.5774404204714162

In [20]:
RMSE

0.8946386461319651

How did it perform on the training set? (This will be used later on for comparison)

In [21]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.528834818302531

### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [22]:
from sklearn.linear_model import RidgeCV

In [ ]:
# help(RidgeCV)

In [68]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [69]:
# The more alpha options you pass, the longer this will take.
# Fortunately our data set is still pretty small
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([1.e-03, 1.e-02, 2.e-02, 3.e-02, 4.e-02, 5.e-02, 7.e-02, 1.e+00,
       1.e+01]),
        scoring='neg_mean_absolute_error')

In [70]:
ridge_cv_model.alpha_

0.03

In [71]:
test_predictions = ridge_cv_model.predict(X_test)

In [64]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [72]:
MAE

0.2914256851708902

In [73]:
RMSE

0.5941140251195123

In [74]:
# Training Set Performance
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.2901312362503258

In [75]:
ridge_cv_model.coef_

array([ 6.43507716,  0.52705718,  0.33934369, -8.61654588,  4.95591969,
       -1.53915625, -1.29091779,  0.70904477,  0.19458354,  3.96348164,
       -1.51058741,  1.26854012,  0.51645572, -0.30047744,  0.08424225,
        0.6531423 , -0.27979037, -0.16265522, -0.18484187])


-----

## Lasso Regression

In [76]:
from sklearn.linear_model import LassoCV

In [78]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.0001,n_alphas=1000,cv=5, max_iter=10000000)

In [79]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.0001, max_iter=10000000, n_alphas=1000)

In [80]:
lasso_cv_model.alpha_

0.0015649387378762383

In [81]:
test_predictions = lasso_cv_model.predict(X_test)

In [82]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [83]:
MAE

0.3859756987522329

In [84]:
RMSE

0.555420477276636

In [85]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.2920387465601074

In [86]:
lasso_cv_model.coef_

array([ 6.49704481,  0.19152026,  0.38707752, -8.84326064,  4.69257829,
       -0.83538534, -0.14594724,  0.13224172, -0.        ,  4.099283  ,
       -1.05233396,  0.62376349,  0.27024306, -0.20913493, -0.        ,
       -0.        , -0.        , -0.        , -0.05210717])

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [102]:
from sklearn.linear_model import ElasticNetCV

In [110]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],eps=0.0001, n_alphas=1000, max_iter=10000000,cv=5)

In [111]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(cv=5, eps=0.0001, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             max_iter=10000000, n_alphas=1000)

In [112]:
elastic_model.l1_ratio_

1.0

In [113]:
test_predictions = elastic_model.predict(X_test)

In [114]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [115]:
MAE

0.3859756987522329

In [116]:
RMSE

0.555420477276636

In [117]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.2920387465601074

In [118]:
elastic_model.coef_

array([ 6.49704481,  0.19152026,  0.38707752, -8.84326064,  4.69257829,
       -0.83538534, -0.14594724,  0.13224172, -0.        ,  4.099283  ,
       -1.05233396,  0.62376349,  0.27024306, -0.20913493, -0.        ,
       -0.        , -0.        , -0.        , -0.05210717])

-----
---